# Модуль 4.6 — RAG по Markdown‑документам

**Цель:** загрузить `.md` файлы из папки, очистить/нормализовать текст и собрать рабочий RAG.

**Что сделаем:**
- установим библиотеки
- создадим примерные `.md` файлы
- распарсим Markdown
- соберем RAG‑пайплайн и зададим вопрос

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  pydantic==2.12.3 \
  requests==2.32.4

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Подготовка папки `md_data`

Если папки нет, создадим её и положим примерные `.md` файлы.

In [ ]:
from pathlib import Path

md_dir = Path("./md_data")
md_dir.mkdir(parents=True, exist_ok=True)

sample_a = md_dir / "intro.md"
if not sample_a.exists():
    sample_a.write_text(
        "# Введение в RAG\n\n"
        "RAG помогает улучшать ответы, используя внешние документы.\n"
        "Он особенно полезен для FAQ и баз знаний.\n",
        encoding="utf-8",
    )

sample_b = md_dir / "chunks.md"
if not sample_b.exists():
    sample_b.write_text(
        "# Чанкинг\n\n"
        "Чанкинг — это разбиение текста на небольшие фрагменты.\n"
        "Перекрытие чанков помогает не терять контекст.\n",
        encoding="utf-8",
    )

print("Файлы в md_data:", [p.name for p in md_dir.glob("*.md")])

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Загрузка и очистка Markdown

Мы загрузим `.md` файлы, удалим Markdown‑разметку и подготовим текст для индексации.

In [ ]:
import re
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Простая очистка Markdown (для демо)
def strip_markdown(text: str) -> str:
    text = re.sub(r"```[\s\S]*?```", " ", text)  # кодовые блоки
    text = re.sub(r"`([^`]*)`", r"\1", text)     # inline code
    text = re.sub(r"#+\s*", "", text)            # заголовки
    text = re.sub(r"\*\*([^*]+)\*\*", r"\1", text)
    text = re.sub(r"\*([^*]+)\*", r"\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

loader = DirectoryLoader(
    "./md_data",
    glob="**/*.md",
    loader_cls=TextLoader,
    show_progress=True,
)

documents = loader.load()

# Применяем очистку к каждому документу
for doc in documents:
    doc.page_content = strip_markdown(doc.page_content)

print("Загружено .md документов:", len(documents))
print("Пример текста:\n", documents[0].page_content)

## Чанкинг → эмбеддинги → векторная база

Собираем индекс для поиска по смыслу.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain.chains import RetrievalQA

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=os.environ.get("OPENAI_BASE_URL"))
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace="openai")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=cached_embedder,
    persist_directory="./db/chroma_md",
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
)

question = "Что такое чанкинг и зачем он нужен?"
answer = rag_chain.invoke({"query": question})
print("Ответ:\n", answer["result"])

## Практические рекомендации

- Для реальных Markdown используйте полноценный парсер (например, `markdown-it`), если важны таблицы/код.
- Удаляйте навигацию и повторяющиеся блоки (toc, footer).
- Храните метаданные: путь, заголовок, раздел документа.